# Hamiltonian Generator

Mizore provides versatile tools in `HamiltonianGenerator` for generating and modifying problem Hamiltonians.

## Molecular Hamiltonians

Mizore produce methods to produce `EnergyObjective` which contains not only the problem Hamiltonian, but also essential information for carrying out VQE on it, such as the Hartree-Fock initialization and energy of chemical accuracy. 

For a simple run or standard tests, the users can use the default molecule Hamiltonians provided by the following functions.

In [22]:
from HamiltonianGenerator import make_example_H6

# Make a default H2 molecular Hamiltonian with sto-3g, and bravyi-kitaev transformation at the equilibrium
energy_obj=make_example_H6()

print("Number of Qubits:",energy_obj.n_qubit)
print("Hartree-Fock init:",energy_obj.init_block)
print("More information:",energy_obj.obj_info)
print("----The Hamiltonian----")
print(energy_obj.hamiltonian)


ImportError: cannot import name 'make_example_H6' from 'HamiltonianGenerator' (/home/copper/Mizore/src/HamiltonianGenerator/__init__.py)

In [16]:
from HamiltonianGenerator import make_example_LiH,make_example_H2O,make_example_N2

# Default: basis:sto-3g active space:2 electron/3 orbital(irrep:A1:3)
energy_obj=make_example_LiH()
print("Number of Qubits:",energy_obj.n_qubit)
# Default: basis:sto-3g active space:4 electron/5 orbital(irrep:B2:2,A1:3)
energy_obj=make_example_H2O()
print("Number of Qubits:",energy_obj.n_qubit)
# Default: basis:cc-pvdz active space:10 electron/8 orbital(irrep:A1:3)
energy_obj=make_example_N2()
print("Number of Qubits:",energy_obj.n_qubit)

Symmetry: Coov  is used when build the molecule.
Number of Qubits: 6
Symmetry: C2v  is used when build the molecule.
Number of Qubits: 10
Symmetry: Dooh  is used when build the molecule.
Number of Qubits: 16


## Graph problem Hamiltonian

Mizore can also run block based qaoa algorithm to solve graph problems, like maxcut and tsp. 

For example, the users can asign a node number to randomly generate a graph to see examples of solving maxcut and tsp problems

In [17]:
from HamiltonianGenerator import make_example_maxcut, make_example_tsp

energy_obj = make_example_maxcut(8)
print("Number of Qubits:",energy_obj.n_qubit)
print("More information:",energy_obj.obj_info)
print("----The Hamiltonian----")
print(energy_obj.hamiltonian)

1 2 None


TypeError: can only concatenate str (not "ABCMeta") to str

In [18]:
import networkx as nx
import matplotlib.pyplot as plt
from HamiltonianGenerator.QAOA._QAOA_from_graph import random_graph, get_maxcut_hamiltonian_from_graph, get_tsp_hamiltonian_from_graph

graph = random_graph(d=3, n=8)
""" for e in graph.edges:
    graph[e[0]][e[1]] = 1.0
    print(graph[e[0],e[1]])
    print(graph.get_edge_data(0, 1)) """
""" pos=nx.kamada_kawai_layout(graph)
plt.figure(figsize=(4, 4))
nx.draw(graph, pos, with_labels=True, font_weight='bold')
plt.show() """
print(graph.edges(data=True))

hamiltonian =  get_maxcut_hamiltonian_from_graph(graph)
print(hamiltonian)


[(2, 7, {}), (2, 1, {}), (2, 4, {}), (7, 5, {}), (7, 0, {}), (1, 4, {}), (1, 5, {}), (4, 6, {}), (6, 0, {}), (6, 3, {}), (5, 3, {}), (0, 3, {})]
2 7 None


TypeError: can only concatenate str (not "ABCMeta") to str

## Stationary qubit reducer